## Code for replicating gao et al research on TroFi Sequence Model

In [1]:
# mount drive
from google.colab import drive
ROOT = '/content/drive'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# add repo directory to path
import os
import sys
from os.path import join 
repo_dir = '/content/drive/MyDrive/Repos/metaphor-detection'
if repo_dir not in sys.path:
    sys.path.append(repo_dir)
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Repos/metaphor-detection']


In [3]:
# directories
# to download glove and elmo vectors see: notebooks/Download_large_data.ipynb
data_dir = repo_dir + '/resources/metaphor-in-context/data/'
glove_dir = repo_dir + '/resources/glove/'
elmo_dir = repo_dir + '/resources/elmo/'

### Gao Code

In [ ]:
# pip install requirements (takes a while)
!cd drive/MyDrive/Repos/metaphor-detection/; pip install -r gao-g-requirements.txt
!pip install --upgrade google-cloud-storage

In [5]:
from core.gao_files.sequence.util import get_num_lines, get_pos2idx_idx2pos, index_sequence, get_vocab, embed_indexed_sequence, \
    get_word2idx_idx2word, get_embedding_matrix, write_predictions, get_performance_VUAverb_val
from core.gao_files.sequence.util import TextDatasetWithGloveElmoSuffix as TextDataset
from core.gao_files.sequence.util import evaluate
from core.gao_files.sequence.model import RNNSequenceModel

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

import csv
import h5py
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [6]:
print("PyTorch version:")
print(torch.__version__)
print("GPU Detected:")
print(torch.cuda.is_available())
using_GPU = torch.cuda.is_available()

PyTorch version:
1.10.0+cu111
GPU Detected:
True


In [7]:
"""
1. Data pre-processing
"""

'''
1.2 TroFi
get raw dataset as a list:
  Each element is a triple:
    a sentence: string
    a index: int: idx of the focus verb
    a label: int 1 or 0
'''
raw_trofi = []

with open(data_dir +'TroFi/TroFi_formatted_all3737.csv') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        sentence = line[1]
        label_seq = [0] * len(sentence.split())
        pos_seq = [0] * len(label_seq)
        verb_idx = int(line[2])
        verb_label = int(line[3])
        label_seq[verb_idx] = verb_label
        pos_seq[verb_idx] = 1   # idx2pos = {0: 'words that are not focus verbs', 1: 'focus verb'}
        raw_trofi.append([sentence.strip(), label_seq, pos_seq])


print('TroFi dataset division: ', len(raw_trofi))


TroFi dataset division:  3737


In [8]:
"""
2. Data preparation
"""
'''
2. 1
get vocabulary and glove embeddings in raw dataset 
'''
# vocab is a set of words
vocab = get_vocab(raw_trofi)
# two dictionaries. <PAD>: 0, <UNK>: 1
word2idx, idx2word = get_word2idx_idx2word(vocab)
# glove_embeddings a nn.Embeddings
glove_embeddings = get_embedding_matrix(glove_dir + 'glove.840B.300d.txt', word2idx, idx2word, normalization=False)
# elmo_embeddings
# set elmos_trofi=None to exclude elmo vectors. Also need to change the embedding_dim in later model initialization
elmos_trofi = h5py.File(elmo_dir + 'TroFi3737.hdf5', 'r')

vocab size:  14881


100%|██████████| 2196017/2196017 [00:59<00:00, 36717.67it/s]


Number of pre-trained word vectors loaded:  14674
Embeddings mean:  0.0016923490911722183
Embeddings stdev:  0.3751399517059326


In [9]:
'''
2. 2
embed the datasets
'''
random.seed(0)
random.shuffle(raw_trofi)

# second argument is the post sequence, which we don't need
embedded_trofi = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                      glove_embeddings, elmos_trofi, None),
                       example[2], example[1]]
                      for example in raw_trofi]

In [ ]:
'''
2. 3 10-fold cross validation
'''
# separate the embedded_sentences and labels into 2 list, in order to pass into the TextDataset as argument
sentences = [example[0] for example in embedded_trofi]
poss = [example[1] for example in embedded_trofi]
labels = [example[2] for example in embedded_trofi]
# ten_folds is a list of 10 tuples, each tuple is (list_of_embedded_sentences, list_of_corresponding_labels)
ten_folds = []
fold_size = int(3737 / 10)
for i in range(10):
    ten_folds.append((sentences[i * fold_size:(i + 1) * fold_size],
                      poss[i * fold_size:(i + 1) * fold_size],
                      labels[i * fold_size:(i + 1) * fold_size]))

idx2pos = {0: 'words that are not focus verbs', 1: 'focus verb'}

optimal_f1s = []
optimal_ps = []
optimal_rs = []
optimal_accs = []
predictions_all = []
for i in range(10):
    '''
    2. 3
    set up Dataloader for batching
    '''
    training_sentences = []
    training_labels = []
    training_poss = []
    for j in range(10):
        if j != i:
            training_sentences.extend(ten_folds[j][0])
            training_poss.extend(ten_folds[j][1])
            training_labels.extend(ten_folds[j][2])
    training_dataset_trofi = TextDataset(training_sentences, training_poss, training_labels)
    val_dataset_trofi = TextDataset(ten_folds[i][0], ten_folds[i][1], ten_folds[i][2])

    # Data-related hyperparameters
    batch_size = 10
    # Set up a DataLoader for the training, validation, and test dataset
    train_dataloader_trofi = DataLoader(dataset=training_dataset_trofi, batch_size=batch_size, shuffle=True,
                                        collate_fn=TextDataset.collate_fn)
    val_dataloader_trofi = DataLoader(dataset=val_dataset_trofi, batch_size=batch_size, shuffle=False,
                                      collate_fn=TextDataset.collate_fn)
    """
    3. Model training
    """
    '''
    3. 1 
    set up model, loss criterion, optimizer
    '''
    # Instantiate the model
    # embedding_dim = glove + elmo + suffix indicator
    # dropout1: dropout on input to RNN
    # dropout2: dropout in RNN; would be used if num_layers=1
    # dropout3: dropout on hidden state of RNN to linear layer
    RNNseq_model = RNNSequenceModel(num_classes=2, embedding_dim=300+1024, hidden_size=300,
                                    num_layers=1, bidir=True,
                                    dropout1=0.5, dropout2=0, dropout3=0.2)
    # Move the model to the GPU if available
    if using_GPU:
        RNNseq_model = RNNseq_model.cuda()
    # Set up criterion for calculating loss
    loss_criterion = nn.NLLLoss()
    # Set up an optimizer for updating the parameters of the rnn_clf
    rnn_optimizer = optim.Adam(RNNseq_model.parameters(), lr=0.001)
    # Number of epochs (passes through the dataset) to train the model for.
    num_epochs = 10

    '''
    3. 2
    train model
    '''
    train_loss = []
    val_loss = []
    performance_matrix = None
    val_f1 = []
    val_p = []
    val_r = []
    val_acc = []
    train_f1 = []
    # A counter for the number of gradient updates
    num_iter = 0
    model_index = 0
    comparable = []
    for epoch in range(num_epochs):
        print("Starting epoch {}".format(epoch + 1))
        for (__, example_text, example_lengths, labels) in train_dataloader_trofi:
            example_text = Variable(example_text)
            example_lengths = Variable(example_lengths)
            labels = Variable(labels)
            if using_GPU:
                example_text = example_text.cuda()
                example_lengths = example_lengths.cuda()
                labels = labels.cuda()
            # predicted shape: (batch_size, seq_len, 2)
            predicted = RNNseq_model(example_text, example_lengths)
            batch_loss = loss_criterion(predicted.view(-1, 2), labels.view(-1))
            rnn_optimizer.zero_grad()
            batch_loss.backward()
            rnn_optimizer.step()
            num_iter += 1
            # Calculate validation and training set loss and accuracy every 200 gradient updates
            if num_iter % 200 == 0:
                avg_eval_loss, performance_matrix = evaluate(idx2pos, val_dataloader_trofi, RNNseq_model,
                                                             loss_criterion, using_GPU)
                val_loss.append(avg_eval_loss)
                val_p.append(performance_matrix[1][0])
                val_r.append(performance_matrix[1][1])
                val_f1.append(performance_matrix[1][2])
                val_acc.append(performance_matrix[1][3])
                print("Iteration {}. Validation Loss {}.".format(num_iter, avg_eval_loss))
#                 avg_eval_loss, performance_matrix = evaluate(idx2pos, train_dataloader_trofi, RNNseq_model,
#                                                              loss_criterion, using_GPU)
#                 train_loss.append(avg_eval_loss)
#                 train_f1.append(performance_matrix[1][1])
#                 print("Iteration {}. Training Loss {}.".format(num_iter, avg_eval_loss))
    print("Training done for fold {}".format(i))

    """
    3.3
    plot the training process: MET F1 and losses for validation and training dataset
    """
#     plt.figure(0)
#     plt.title('F1 for TroFI dataset on fold ' + str(i))
#     plt.xlabel('iteration (unit:200)')
#     plt.ylabel('F1')
#     plt.plot(val_f1, 'g')
#     #     plt.plot(train_f1, 'b')
#     plt.legend(['Validation F1', 'Training F1'], loc='upper right')
#     plt.show()

#     plt.figure(1)
#     plt.title('Loss for TroFi dataset on fold ' + str(i))
#     plt.xlabel('iteration (unit:200)')
#     plt.ylabel('Loss')
#     plt.plot(val_loss, 'g')
#     #     plt.plot(train_loss, 'b')
#     plt.legend(['Validation loss', 'Training loss'], loc='upper right')
#     plt.show()

    """
    store the best f1
    """
    print('val_f1: ', val_f1)
    idx = 0
    if math.isnan(max(val_f1)):
        optimal_f1s.append(max(val_f1[6:]))
        idx = val_f1.index(optimal_f1s[-1])
        optimal_ps.append(val_p[idx])
        optimal_rs.append(val_r[idx])
        optimal_accs.append(val_acc[idx])
    else:
        optimal_f1s.append(max(val_f1))
        idx = val_f1.index(optimal_f1s[-1])
        optimal_ps.append(val_p[idx])
        optimal_rs.append(val_r[idx])
        optimal_accs.append(val_acc[idx])

In [12]:
"""
print out the performance
plot the performance on each fold
"""
print('F1 on TroFi by 10-fold = ', optimal_f1s)
print('Precision on TroFi = ', np.mean(np.array(optimal_ps)))
print('Recall on TroFi = ', np.mean(np.array(optimal_rs)))
print('F1 on TroFi = ', np.mean(np.array(optimal_f1s)))
print('Accuracy on TroFi = ', np.mean(np.array(optimal_accs)))

F1 on TroFi by 10-fold =  [74.82993197278911, 71.25, 68.96551724137932, 68.24925816023739, 68.85245901639344, 70.93023255813954, 70.15384615384616, 72.38095238095237, 71.9242902208202, 71.21661721068249]
Precision on TroFi =  70.44295206262248
Recall on TroFi =  71.46397164548841
F1 on TroFi =  70.875310491524
Accuracy on TroFi =  74.36997319034852


In [24]:
import pandas as pd
gao_scores = [70.7, 71.6, 71.1, 74.6]
our_scores = [np.mean(np.array(optimal_ps)),
  np.mean(np.array(optimal_rs)),
  np.mean(np.array(optimal_f1s)),
  np.mean(np.array(optimal_accs))]
our_scores = [round(score,1) for score in our_scores]
all_scores = [gao_scores, our_scores]
all_scores_df = pd.DataFrame(all_scores, columns= ['P', 'R', 'F1', 'Acc'], index=['Gao et al', 'US'])
print("trofi seq model: classification task\n")
all_scores_df

trofi seq model: classification task



,P,R,F1,Acc
Gao et al,70.7,71.6,71.1,74.6
US,70.4,71.5,70.9,74.4
